In [1]:
%reload_ext autoreload
%autoreload 2

import sys

import numpy as np
import healpy as hp
import jax.numpy as jnp
from jax import jit, vmap
    
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import colormaps as cms
mpl.rc_file('../matplotlibrc')

In [ ]:
sys.path.append("..")

In [3]:
valid_snr_list = load_snr_list("../outputs/snr/prebuild_snr_list_fullinfo.json")

## 1. Location

## 1. Location

## 1. Location

## 1. Location

## 1. Location

## 1. Location